In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import time

In [ ]:
filepath ='data/'
csvfilenames = ['confinement_training.csv', 'member_information.csv', 'labresults_training.csv', 'medical_training.csv', 'rx_training.csv']
txtfilenames = ['lu_diagnosis.txt', 'lu_procedure-fake.txt', 'lu_ndc.txt' ]

with open(filepath+csvfilenames[0]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

confinement = pd.DataFrame(data)
confinement.columns = confinement.iloc[0]
confinement = confinement[1:] #sets first row as header

with open(filepath+csvfilenames[1]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open(filepath+csvfilenames[3]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

allmedical = pd.DataFrame(data)
allmedical.columns = allmedical.iloc[0]
allmedical = allmedical[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

In [ ]:
allmedical['STD_COST']=allmedical['STD_COST'].apply(pd.to_numeric) #change entries to numbers
rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers

In [ ]:
#completely unnecessary for faster version of code below
members = members.sort_values(['PATID'])
rxdata = rxdata.sort_values(['PATID'])
allmedical = allmedical.sort_values(['PATID'])

In [ ]:
#slower version - don't run
tick = time.clock()
members['totalcost']=0
print len(members)
print len(rxdata)
k=0
j=0
while k < len(members):
    while j < len(rxdata) and rxdata.iloc[j]['PATID']==members.iloc[k].name:
        members.iloc[k,3] += rxdata.iloc[j]['STD_COST']
        j+=1
    k+=1
    if k % 1000 == 0:
        print k
        tock = time.clock()
        print tock-tick
    
print j, len(rxdata), k, len(members)

In [ ]:
#faster version
tick = time.clock()
members['totalcost']=0
print len(members)
print len(rxdata)
k=0
while k < len(rxdata):
    members.loc[rxdata.iloc[k]['PATID'],'totalcost']+=rxdata.iloc[k]['STD_COST']
    k += 1
    if k % 60000 == 0:
        print k
        tock=time.clock()
        print tock-tick

In [ ]:
ser = allmedical['STD_COST']
ser.plot.density()
plt.xlim([-5000,5000])
plt.show()

In [ ]:
allmedical.sort_values(["STD_COST"], ascending = [1])